In [1]:
func_act = "softmax"
func_costo = "EC"

entrenamiento = 0.7
alpha = 0.1
#en costo poner 'ECM' o 'EC_binaria' o 'EC'

################

%matplotlib inline
import pandas as pd
import numpy as np
from sklearn import preprocessing, metrics, model_selection

import time
#from matplotlib import pyplot as plt
from matplotlib import pylab as plt
from IPython import display

from grafica import *
from ClassRNMulticlase import RNMulticlase

datos = pd.read_csv('../../Datos/Vinos.csv', sep=";")
#datos
#print(datos.Class.unique())

reps = 15
sumaAccTrain = 0
sumaAccTest = 0
sumaIteraciones = 0

print("iteraciones, acc train, acc test: ")

for rep in range(reps):
    testeo = 1 - entrenamiento

    X = np.array(datos.iloc[:, 1:])
    T = np.array(datos['Class'])

    enc = preprocessing.OneHotEncoder(handle_unknown='ignore')
    T = enc.fit_transform(T.reshape(-1,1)).toarray()

    #print(T)#hacemos una columna para cada clase posible
    
    X_train, X_test, T_train, T_test = model_selection.train_test_split(
    X, T, test_size=testeo)

    normalizador= preprocessing.StandardScaler() #media y desvío
    X_train = normalizador.fit_transform(X_train)
    X_test  = normalizador.transform(X_test)
    
    if (func_act == 'tanh'):
        T_train = 2*T_train-1
        T_test  = 2*T_test-1

    rn = RNMulticlase(alpha=alpha, n_iter=400, cotaE=1e-06, FUN=func_act, COSTO=func_costo, random_state=None)
    resul = rn.fit(X_train, T_train)
    
#     plt.plot(range(1, len(rn.errors_) + 1), rn.errors_, marker='o')
#     plt.xlabel('Iteraciones')
#     plt.ylabel(func_costo)
#     plt.show()

    iteraciones = len(rn.errors_)
    sumaIteraciones += iteraciones
    
#     plt.plot(range(1, len(rn.accuracy_) + 1), rn.accuracy_, marker='o')
#     plt.xlabel('Iteraciones')
#     plt.ylabel('accuracy')
#     plt.show()
    
    #-- accuracy en entrenamiento y testeo ---
    acc_Train = rn.accuracy(X_train, T_train)
    acc_Test = rn.accuracy(X_test, T_test)
    
    print(f"{iteraciones},    {acc_Train},    {acc_Test}")
    
    sumaAccTrain += acc_Train
    sumaAccTest += acc_Test

print("-------------------------")
print("PROMEDIOS:")
print("iteraciones: ",sumaIteraciones/reps, " acc train: ", sumaAccTrain/reps, " acc test: ", sumaAccTest/reps)


iteraciones, acc train, acc test: 
400,    0.9838709677419355,    0.9259259259259259
389,    0.967741935483871,    0.9814814814814815
400,    1.0,    0.8518518518518519
270,    0.967741935483871,    0.9074074074074074
400,    0.9516129032258065,    0.9444444444444444
389,    0.9758064516129032,    0.9259259259259259
400,    0.9838709677419355,    0.9259259259259259
321,    0.9838709677419355,    0.9444444444444444
346,    0.967741935483871,    0.9259259259259259
341,    0.9758064516129032,    0.9814814814814815
372,    0.9838709677419355,    0.8888888888888888
400,    1.0,    0.9444444444444444
400,    0.9758064516129032,    0.9444444444444444
400,    0.9838709677419355,    0.9444444444444444
400,    0.9758064516129032,    0.9259259259259259
-------------------------
PROMEDIOS:
iteraciones:  375.2  acc train:  0.9784946236559141  acc test:  0.9308641975308642


# Resultados:
para no perder tanto tiempo siempre uso alpha 0.2, entrenamiento del 70%, haciendo 15 iteraciones para cada variante. Todos los datos son el promedio de las 15 ejecuciones hechas con esa configuración

|función activación|función costo|iteraciones|acc train|acc test|
|---|---|---|---|---|
|sigmoide|ecm|400|0.988|0.927
|sigmoide|ec_bin|333.333|0.989|0.938
|tanh|ecm|391.8|0.988|0.933
|softmax|ec|375.2|0.978|0.931

# Conclusiones:
la función sigmoide y tangencial no tienen mucha diferencia cuando ambas usan la función de costo de error mínimo cuadrado, pero la sigmoideal con error binario es lo que mejor funciona. Yo esperaba que al usar softmax los resultados se entrenen más rápido y que tengan una mejor precisión, pero parece que esto segundo no es así.
La utilidad de usar softmax es que la suma de los resultados de predecir entre diferentes clases suma 1, no tiene porqué tener mejores resultados. Al igual que el ej anterior vemos que sigmoide se entrena más rápiod con ec_bin. Softmax permite que diga "esta clase tiene X probabilidad de ser esta clase A", esto no lo podemos hacer con las otras funciones